In [61]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import kaleido
# Hugging Face Transformers and datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, pipeline
from datasets import load_dataset, Dataset

# scikit-learn for dataset splitting
from sklearn.model_selection import train_test_split



this first cell is for loading my dataset, splitting it into a test and training set. then saving the respective datasets to csv files

In [62]:
dataset = load_dataset("mediabiasgroup/BABE-v3")
df = pd.DataFrame(dataset["train"])
DF_TRAIN, DF_TEST = train_test_split(df, test_size=0.20, random_state=42)
DF_TRAIN.to_csv("TRAINING_DATAFRAME.csv", index=False)
DF_TEST.to_csv("TESTING_DATAFRAME.csv", index=False)

This is for running the pre trained models

In [63]:
#This is for the analysis of the D1V1DE bias-detection model (https://huggingface.co/D1V1DE/bias-detection?text=I+like+you.+I+love+you)
def PRED1V1DE():
    try:
        pipe = pipeline("text-classification", model="D1V1DE/bias-detection")
        CurrentDF = pd.read_csv("TESTING_DATAFRAME.csv")
        CurrentDF['Predicted'] = 'XXX'
        CurrentDF.drop(['news_link','outlet','label','label_opinion','biased_words'], axis=1, inplace=True)
        for index, row in CurrentDF.iterrows():
            text_data = row['text']
            bias = pipe(text_data)
            CurrentDF.at[index, 'Predicted'] = bias[0]["label"]
        CurrentDF.to_csv("temp.csv", index = False)
        
        Type =  ScorePerTopic(CurrentDF, "type")
        Topic = ScorePerTopic(CurrentDF, "topic")
        results = Type.join(Topic).T
        results.to_csv("test.csv", index=True)
        print("D1V1DE/bias-detection examination completed successfully")
        return results
    except Exception as e:
        print("D1V1DE/bias-detection failed")
        print(e)


the bellow section is for fine tuning models and their evaluations

In [64]:
#D1V1DE/bias-detection fine tuning and analysis

#fine tuning the model with my training dataset
def FIND1V1DE(training_data_path):
    # Load and preprocess the dataset
    df = pd.read_csv(training_data_path)
    df = df[['text', 'label']]
    df_train, df_val = train_test_split(df, test_size=0.1)
    train_dataset = Dataset.from_pandas(df_train)
    val_dataset = Dataset.from_pandas(df_val)
    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained("D1V1DE/bias-detection")
    # Tokenization function
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)
    # Tokenize the datasets
    tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
    tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)
    # Load the pretrained model
    model = AutoModelForSequenceClassification.from_pretrained("D1V1DE/bias-detection")
    # Training arguments
    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10
    )
    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_val_dataset
    )
    # Fine-tune the model
    trainer.train()
    # Save the fine-tuned model
    model.save_pretrained("./fine_tuned/D1V1DE")



In [65]:
#code for fine-tuning individual models
#FIND1V1DE("TRAINING_DATAFRAME.csv")

Bellow is all nececary code for evaluations

In [66]:
#used for doing all my scoring and accuracy testing of models using the bias dataset
#variables need renaming and potentailly this code could be sped up
def ScorePerTopic(data, field):
    unique_items = data[field].unique().tolist()
    scores = {item: {"correct": 0, "count": 0} for item in unique_items}
    for index, row in data.iterrows():
        correct = False
        if pd.isna(row[field]) and row['Predicted'] == 'NEUTRAL':
            correct = True
        elif not pd.isna(row[field]) and row['Predicted'] != 'NEUTRAL':
            correct = True
        scores[row[field]]['count'] += 1
        if correct:
            scores[row[field]]['correct'] += 1
    current = {}
    for item in scores:
        scores[item]['score'] = scores[item]['correct'] / scores[item]['count']
    scores = pd.DataFrame(scores).T
    scores["field"] = field #could do with changing from field
    return pd.DataFrame(scores).T


In [67]:


def evaluate_model(model, tokenizer, eval_dataset):
    # Initialize the trainer
    trainer = Trainer(model=model)
    # Tokenize the evaluation dataset
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)
    tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)
    # Evaluate the model
    results = trainer.evaluate(tokenized_eval_dataset)
    return results

using the standard python tools for evaluating models performance

In [68]:
#evalating fine tuned D1V1DE model
tokenizer = AutoTokenizer.from_pretrained("D1V1DE/bias-detection")
model = AutoModelForSequenceClassification.from_pretrained("fine_tuned/D1V1DE")
df_eval = pd.read_csv('TESTING_DATAFRAME.csv')
eval_dataset = Dataset.from_pandas(df_eval[['text', 'label']])
# Evaluate the model
evaluation_results = evaluate_model(model, tokenizer, eval_dataset)

print("FIND1V1DE: ", evaluation_results)


# Replace 'D1V1DE/original-model-name' with the correct path to the original model on Hugging Face
original_model = AutoModelForSequenceClassification.from_pretrained("D1V1DE/bias-detection")
# Evaluate the original model
original_evaluation_results = evaluate_model(original_model, tokenizer, eval_dataset)

# Print the evaluation results of the original model
print("PRED1V1DE: ", original_evaluation_results)


/Users/oscar/miniforge3/envs/oscar/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
100%|██████████| 104/104 [00:17<00:00,  6.09it/s]


FIND1V1DE:  {'eval_loss': 0.38539549708366394, 'eval_runtime': 17.0939, 'eval_samples_per_second': 48.263, 'eval_steps_per_second': 6.084}


100%|██████████| 104/104 [00:16<00:00,  6.31it/s]

PRED1V1DE:  {'eval_loss': 0.29630759358406067, 'eval_runtime': 16.4991, 'eval_samples_per_second': 50.003, 'eval_steps_per_second': 6.303}


bellow is code for visualisations and evaluation assistive tools

In [69]:
def star(data,key):
    FilteredData = data[data['field'] == key]
    # Preparing the data for the star plot (radar chart)
    FilteredData = FilteredData.reset_index()
    FilteredData['index'] = FilteredData['index'].fillna('No Bias')
    labels=FilteredData['index']
    #print(labels)
    stats=FilteredData['score']
    #print(stats)

    # Create radar chart
    angles=np.linspace(0, 2*np.pi, len(labels), endpoint=False).tolist()
    stats=np.concatenate((stats,[stats[0]]))
    angles+=angles[:1]
    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    ax.fill(angles, stats, color='blue', alpha=0.25)
    ax.set_yticklabels([])
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels)

    # Display the plot
    plt.title('Star Plot of '+  str(key) +' vs. accuracy rating')
    plt.show()